## Importing libraries

In [2]:
import pandas as pd # to store data efficiently
import numpy as np # to handle numerical operations
import random # to generate random numbers
import os # to handle operations regarding file system
import wget # to download locally images
from ultralytics import YOLO # to import YOLO model
from tqdm import tqdm # to visualize loop progressions
import matplotlib.pyplot as plt # to create visualizations
import cv2 # to handle images
import re # to identify regu


## Getting images list with people from COCO dataset

In [3]:
anns_df = pd.read_csv('./data/coco_diff/single_anns/person_anns.csv')

In [4]:
img_url_list = list(anns_df['img_url'].unique())
print(len(img_url_list))

58883


## Random sampling from that list and download locally images

In [5]:
random.seed(24) # to use same collection of images
img_sample = random.sample(img_url_list, 2000) 

### Downloads images inside list

In [6]:
# Changes cwd (to download image via wget in) and then changes it back
old_cwd = os.getcwd()
new_cwd = old_cwd+'/imgs'
os.chdir(new_cwd)
for url in tqdm(img_sample):
    try:
        file_name = url.split('/')[-1]
        file_path = os.path.join(new_cwd, file_name)
        
        if not os.path.exists(file_path):
            wget.download(url)
    except Exception as e:
        print(f"\nError in download of {url}: {e}")
os.chdir(old_cwd)

  0%|          | 0/2000 [00:00<?, ?it/s]

100%|██████████| 2000/2000 [00:00<00:00, 8447.14it/s]


In [7]:
model = YOLO("./models/YOLO/yolo11n.pt")

In [8]:
def predict_image_cropped(source: str, crop_part: str = None, crop_factor: int = None, fill: bool = False):
    for filename in tqdm(os.listdir(source)):
        if filename.endswith(('.jpg', 'jpeg', '.png')):
            image_path = os.path.join(source, filename)
            
            image = cv2.imread(image_path)
            
            if image is None:
                print(f"Error in image loading: {filename}")
                continue
            
            # Determine if the image is black & white or RGB
            if np.all(image[..., 0] == image[..., 1]) and np.all(image[..., 1] == image[..., 2]):
                format = 'b&w'
            else:
                format = 'rgb'
            
            height, width, _ = image.shape
            
            if crop_part and crop_factor:
                mask = np.zeros_like(image)  # Create a black image of the same shape
                if fill == "red":
                    fill_color = (0, 0, 255)  # BGR format for red
                elif fill == "white":
                    fill_color = (255, 255, 255)  # BGR format for white
                elif fill == "black" or fill is None:
                    fill_color = (0, 0, 0)  # BGR format for black
                else:
                    print(f"Invalid fill color: {fill}. Use 'black', 'red', or 'white'.")
                    continue
                
                if fill:  # Fill the non-cropped regions with the specified color
                    if crop_part == "top":
                        crop_height = height * crop_factor // 100
                        mask[:crop_height, :, :] = image[:crop_height, :, :]  # Copy only the top part
                        mask[crop_height:, :, :] = fill_color  # Fill the rest
                    elif crop_part == "side":
                        crop_width = width * crop_factor // 100
                        mask[:, :crop_width, :] = image[:, :crop_width, :]  # Copy only the left side
                        mask[:, crop_width:, :] = fill_color  # Fill the rest
                    else:
                        print(f"Invalid crop part for fill: {crop_part}. Use 'top' or 'side'.")
                        continue
                    input_image = mask  # Keep only the filled part
                else:  # Default cropping behavior
                    if crop_part == "top":
                        input_image = image[:height * crop_factor // 100, :, :]
                    elif crop_part == "side":
                        input_image = image[:, :width * crop_factor // 100, :]
                    else:
                        print(f"Invalid crop: {crop_part}. Use 'top' or 'side'.")
                        continue
            else: 
                input_image = image
            
            # Model prediction
            result = model.predict(source=[input_image], verbose=False)
            yield [result, image_path, format]


In [9]:
df_names = ['All', 'Top25', 'Top50', 'Top75', 'Side25', 'Side50', 'Side75']

### Get confidence score for both all and cropped images

In [10]:
def get_img_info(res, img_id: str, format: str, detection_list: list) -> None:
    # Get mask for class 0
        class_mask = res.boxes.cls == 0
        
        # Get all relevant data in one go
        confidences = res.boxes.conf[class_mask]
        dimensions = res.boxes.xywhn[class_mask]
        
        # Print results using zip to iterate both arrays simultaneously
        for index, (conf, dim) in enumerate(zip(confidences, dimensions)):
            detection_dict = {}  # Crea un nuovo dizionario per ogni rilevamento
            detection_dict['Img_ID'] = img_id
            detection_dict['Confidence'] = conf.item()
            detection_dict['Dimensions'] = dim.tolist()
            detection_dict['ID'] = f'{img_id}{index}'
            detection_dict['Format'] = format
            detection_list.append(detection_dict)
        
        # Add specific case if no person is detected
        if len(confidences) == 0:
            detection_dict = {  
                'Img_ID': img_id,
                'Confidence': 0,
                'Dimensions': [],
                'ID': img_id,
                'Format': format
            }
            detection_list.append(detection_dict)

In [11]:
def confidence_score(result_img, cropped: bool = False) -> list:
    detection_list = []
    for res in result_img:
        format = res[2]  # 'b&w' or 'rgb'
        if cropped:
            img_id = res[1].split('\\')[-1].strip('.jpg').lstrip('0')
            res = res[0][0]
        else:
            img_id = res.path.split('\\')[-1].strip('.jpg').lstrip('0')
        get_img_info(res, img_id, format, detection_list)
    return detection_list


In [12]:
img_source = './imgs'

# predicts_obj_list = []
# for name in df_names:
#     print(name)
#     if bool(re.search(r'\d', name)):
#         direction = (name[:-2]).lower() or None 
#         crop_factor = int(name[-2:]) or None
#         predict_raw = list(predict_image_cropped(img_source, direction, crop_factor))
#     else:
#         predict_raw = list(predict_image_cropped(img_source))        
#     predicts_obj_list.append(confidence_score(predict_raw, cropped=True))


In [13]:
# predicts_filled_list = []
# for name in df_names:
#     if bool(re.search(r'\d', name)):
#         direction = (name[:-2]).lower() or None 
#         crop_factor = int(name[-2:]) or None
#         predict_raw = list(predict_image_cropped(img_source, direction, crop_factor, fill='black'))
#         predicts_filled_list.append(confidence_score(predict_raw, cropped=True))
        

In [21]:
predicts_filled_red_list = []
for name in df_names:
    if bool(re.search(r'\d', name)):
        direction = (name[:-2]).lower() or None 
        crop_factor = int(name[-2:]) or None
        predict_raw = list(predict_image_cropped(img_source, direction, crop_factor, fill='red'))
        predicts_filled_red_list.append(confidence_score(predict_raw, cropped=True))
        

  0%|          | 0/2001 [00:00<?, ?it/s]

100%|██████████| 2001/2001 [07:18<00:00,  4.56it/s]


In [22]:
predicts_filled_white_list = []
for name in df_names:
    if bool(re.search(r'\d', name)):
        direction = (name[:-2]).lower() or None 
        crop_factor = int(name[-2:]) or None
        predict_raw = list(predict_image_cropped(img_source, direction, crop_factor, fill='white'))
        predicts_filled_white_list.append(confidence_score(predict_raw, cropped=True))
        

100%|██████████| 2001/2001 [05:59<00:00,  5.56it/s]


In [16]:
# pred_df_list = []
# for obj, name in zip(predicts_obj_list, df_names):
#     df = pd.DataFrame(obj).set_index('ID')
#     df['x_center'] = df['Dimensions'].apply(lambda x: x[0] if x else np.nan)
#     df['y_center'] = df['Dimensions'].apply(lambda x: x[1] if x else np.nan)
#     df['width'] = df['Dimensions'].apply(lambda x: x[2] if x else np.nan)
#     df['height'] = df['Dimensions'].apply(lambda x: x[3] if x else np.nan)
#     df = df.drop(['Dimensions'], axis=1)
#     pred_df_list.append(df)
#     df.to_csv(f'./data/yolo_prediction/{name}.csv', index=True)

In [17]:
# pred_df_list = []
# for obj, name in zip(predicts_filled_list, df_names[1:]):
#     df = pd.DataFrame(obj).set_index('ID')
#     df['x_center'] = df['Dimensions'].apply(lambda x: x[0] if x else np.nan)
#     df['y_center'] = df['Dimensions'].apply(lambda x: x[1] if x else np.nan)
#     df['width'] = df['Dimensions'].apply(lambda x: x[2] if x else np.nan)
#     df['height'] = df['Dimensions'].apply(lambda x: x[3] if x else np.nan)
#     df = df.drop(['Dimensions'], axis=1)
#     pred_df_list.append(df)
#     df.to_csv(f'./data/yolo_prediction/{name}_filled.csv', index=True)

In [23]:
pred_df_list = []
for obj, name in zip(predicts_filled_red_list, df_names[1:]):
    df = pd.DataFrame(obj).set_index('ID')
    df['x_center'] = df['Dimensions'].apply(lambda x: x[0] if x else np.nan)
    df['y_center'] = df['Dimensions'].apply(lambda x: x[1] if x else np.nan)
    df['width'] = df['Dimensions'].apply(lambda x: x[2] if x else np.nan)
    df['height'] = df['Dimensions'].apply(lambda x: x[3] if x else np.nan)
    df = df.drop(['Dimensions'], axis=1)
    pred_df_list.append(df)
    df.to_csv(f'./data/yolo_prediction/{name}_red.csv', index=True)

In [24]:
pred_df_list = []
for obj, name in zip(predicts_filled_white_list, df_names[1:]):
    df = pd.DataFrame(obj).set_index('ID')
    df['x_center'] = df['Dimensions'].apply(lambda x: x[0] if x else np.nan)
    df['y_center'] = df['Dimensions'].apply(lambda x: x[1] if x else np.nan)
    df['width'] = df['Dimensions'].apply(lambda x: x[2] if x else np.nan)
    df['height'] = df['Dimensions'].apply(lambda x: x[3] if x else np.nan)
    df = df.drop(['Dimensions'], axis=1)
    pred_df_list.append(df)
    df.to_csv(f'./data/yolo_prediction/{name}_white.csv', index=True)